In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from seysmo.models.model_class import CNN, CNNLSTMNetwork, CNNLSTMNetwork2
from seysmo.models.train_model import train, evaluate
from seysmo.models.utils import save_model, load_model, count_parameters, EarlyStopper, SignalSpeedDataset, give_data
from seysmo.visualization.plotting import plot_map
from seysmo.features.mapping import do_array_for_mapping, compute_y_pred
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchinfo import summary
from torchmetrics import MeanAbsolutePercentageError

import mlflow

mlflow.set_tracking_uri("http://localhost:5000")

In [2]:
X_train, y_train, coord_train, X_val, y_val, coord_val, X_test, y_test, coord_test = give_data(
    '../data/processed/coord_power_dict.pkl')

In [3]:
X_train.shape

(11045, 500, 64)

In [4]:
X_train[0]

array([[0.03285914, 0.21393382, 0.0712524 , ..., 0.00570705, 0.04982899,
        0.0083305 ],
       [0.08130122, 0.23208489, 0.03740172, ..., 0.0085214 , 0.00848223,
        0.04383594],
       [0.15692293, 0.21502408, 0.03025572, ..., 0.04717113, 0.00122513,
        0.02647391],
       ...,
       [0.47926048, 0.39923518, 0.35052804, ..., 0.79563634, 0.79361495,
        0.75849848],
       [0.47943677, 0.39941341, 0.35074422, ..., 0.77769772, 0.77557886,
        0.73915492],
       [0.47961504, 0.39959407, 0.35096377, ..., 0.75935655, 0.75715423,
        0.71949755]])

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
in_channels = 1
out_channels = 16
conv_kernel_size = 5
stride = 2
padding = 1
hidden_size = 128
num_layers = 16
dropout = 0.2
linear_input_size = 64
linear_output_size = 10
input_shape = X_train[0].shape
epochs = 250
batch_size = 32
learning_rate = 0.001
patience = 3
min_delta = 50
loss_fn = nn.MSELoss()
metric_fn = MeanAbsolutePercentageError().to(device)
model0 = CNNLSTMNetwork(in_channels=in_channels, out_channels=out_channels, kernel_size=conv_kernel_size, stride=stride,
                        padding=padding, hidden_size=hidden_size, num_layers=num_layers, dropout=dropout,
                        linear_input_size=linear_input_size, linear_output_size=linear_output_size,
                        input_shape=input_shape).to(device)
early_stopper = EarlyStopper(patience=patience, min_delta=min_delta)
optimizer = torch.optim.Adam(model0.parameters(), lr=learning_rate)

In [6]:
train_dataset = SignalSpeedDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = SignalSpeedDataset(X_val, y_val)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_dataset = SignalSpeedDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [7]:
mlflow.set_experiment("/CNNLSTM_v0.1")

<Experiment: artifact_location='mlflow-artifacts:/536689916772439481', creation_time=1721906761771, experiment_id='536689916772439481', last_update_time=1721906761771, lifecycle_stage='active', name='/CNNLSTM_v0.1', tags={}>

In [8]:
with mlflow.start_run() as run:
    params = {
        "epochs": epochs,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "loss_function": loss_fn.__class__.__name__,
        "metric_function": metric_fn.__class__.__name__,
        "optimizer": "ADAM",
        "in_channels": in_channels,
        'out_channels': out_channels,
        "conv_kernel_size": conv_kernel_size,
        'stride': stride,
        'padding': padding,
        'hidden_size': hidden_size,
        'num_layers': num_layers,
        'dropout': dropout,
        'linear_input_size': linear_input_size,
        'linear_output_size': linear_output_size,
        "input_shape": input_shape,
        'patience': patience,
        'min_delta': min_delta
    }
    # Log training parameters.
    mlflow.log_params(params)

    # Log model summary.
    with open("../models/model_P_summary.txt", "w", encoding="utf-8") as f:
        f.write(str(summary(model0)))
    mlflow.log_artifact("../models/model_P_summary.txt")

    for t in range(epochs):
        print(f"Epoch {t + 1}\n-------------------------------")
        train(train_dataloader, model0, loss_fn, metric_fn, optimizer, epoch=t, device=device,
              output_size=linear_output_size)
        validation_loss, validation_mape = evaluate(val_dataloader, model0, loss_fn, metric_fn, epoch=t, device=device,
                                                    output_size=linear_output_size)

    # Save the trained model to MLflow.
    mlflow.pytorch.log_model(model0, "model")

Epoch 1
-------------------------------
loss: 133569.390625 MAPE: 1.000016 [0 / 346]
loss: 92686.757812 MAPE: 0.813889 [100 / 346]
loss: 24642.785156 MAPE: 0.303440 [200 / 346]
loss: 6879.498535 MAPE: 0.170452 [300 / 346]
Eval metrics: 
MAPE: 0.15, Avg loss: 4060.181864 

Epoch 2
-------------------------------
loss: 4551.247070 MAPE: 0.158465 [0 / 346]
loss: 3778.647705 MAPE: 0.143768 [100 / 346]
loss: 3153.325928 MAPE: 0.136358 [200 / 346]
loss: 3706.957520 MAPE: 0.152920 [300 / 346]
Eval metrics: 
MAPE: 0.14, Avg loss: 3488.053467 

Epoch 3
-------------------------------
loss: 4180.910156 MAPE: 0.134392 [0 / 346]
loss: 2795.520752 MAPE: 0.125345 [100 / 346]
loss: 4003.223877 MAPE: 0.131220 [200 / 346]
loss: 3129.892578 MAPE: 0.142199 [300 / 346]
Eval metrics: 
MAPE: 0.14, Avg loss: 3487.741494 

Epoch 4
-------------------------------
loss: 4735.196289 MAPE: 0.161724 [0 / 346]
loss: 5202.297852 MAPE: 0.183021 [100 / 346]
loss: 3937.070801 MAPE: 0.144563 [200 / 346]
loss: 2905.54003

2024/08/01 11:27:59 WARNING mlflow.utils.requirements_utils: Found torch version (2.3.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.3.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/08/01 11:28:14 WARNING mlflow.utils.requirements_utils: Found torch version (2.3.1+cu118) contains a local version label (+cu118). MLflow logged a pip requirement for this package as 'torch==2.3.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.


In [9]:
num_batches = len(test_dataloader)
eval_loss, eval_mape = 0, 0
with torch.no_grad():
    for X, y in test_dataloader:
        X, y = X.to(device), y.to(device)
        pred = model0(X)
        eval_loss += loss_fn(torch.reshape(pred, (-1, 10)), torch.reshape(y, (-1, 10))).item()
        eval_mape += metric_fn(torch.reshape(pred, (-1, 10)), torch.reshape(y, (-1, 10))).item()

eval_loss /= num_batches
eval_mape /= num_batches
print(f"Test loss: {eval_loss:4f}")
print(f"Test MAPE: {eval_mape:4f}")

Test loss: 3457.489180
Test MAPE: 0.138088
